In [2]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import os
import pandas as pd
import pyodbc

Change the filepath to where JSON authentication file is located

In [3]:
path = 'Downloads'
os.chdir(path)

Code to connect to the Google Drive API

In [4]:
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('CovidData-63b330342298.json', scope)
client = gspread.authorize(creds)

Open the Workbook named "Covid_Tracker" on Google Sheets

In [5]:
sheet = client.open('Covid_Tracker')

Connect to two sheets within the Covid_Tracker workbook, Cases & Deaths

In [6]:
CasesSheet = sheet.worksheet("Cases")
DeathsSheet = sheet.worksheet("Deaths")

Connect to local SQL database hosting the Covid Data

In [7]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=LAPTOP-4KUL2T84\SQLEXPRESS;'
                      'Database=Covid;'
                      'Trusted_Connection=yes;')

Pull down two sql queries and convert them into pandas dataframes

In [8]:
Cases_Query = pd.read_sql_query(

'''
    SELECT Cases, Ward, Date
    FROM Cases
    Inner JOIN Wards ON Cases.WardID=Wards.WardID;
'''
    , conn)
dfCasesPull = pd.DataFrame(Cases_Query)

Deaths_Query = pd.read_sql_query(

'''
    SELECT Deaths, Ward, Date
    FROM Deaths
    Inner JOIN Wards ON Deaths.WardID=Wards.WardID;
'''
    , conn)
dfDeathsPull = pd.DataFrame(Deaths_Query)


Push these respective dataframes into Google Sheets

In [9]:
DeathsSheet.update([dfDeathsPull.columns.values.tolist()] + dfDeathsPull.values.tolist())
CasesSheet.update([dfCasesPull.columns.values.tolist()] + dfCasesPull.values.tolist())


{'spreadsheetId': '1le3E0imbMVbxxTZyKsbCxMxfIg4_o3O9uCvgzXDroFc',
 'updatedRange': 'Cases!A1:C469',
 'updatedRows': 469,
 'updatedColumns': 3,
 'updatedCells': 1407}